In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
from functools import lru_cache
import time

def demonstrate_partial_derivative():
    # Mathematical functions with caching
    @lru_cache(maxsize=128)
    def f(x, y, func_type):
        if func_type == 'paraboloid':
            return x**2 + y**2
        elif func_type == 'saddle':
            return x**2 - y**2
        elif func_type == 'wave':
            return np.sin(x) + np.cos(y)
    
    @lru_cache(maxsize=128)
    def df_dx(x, y, func_type):
        if func_type == 'paraboloid':
            return 2*x
        elif func_type == 'saddle':
            return 2*x
        elif func_type == 'wave':
            return np.cos(x)
    
    @lru_cache(maxsize=128)
    def df_dy(x, y, func_type):
        if func_type == 'paraboloid':
            return 2*y
        elif func_type == 'saddle':
            return -2*y
        elif func_type == 'wave':
            return -np.sin(y)
    
    # Data generator with caching
    @lru_cache(maxsize=32)
    def generate_data(x_min, x_max, y_min, y_max, res, func_type):
        x = np.linspace(x_min, x_max, res)
        y = np.linspace(y_min, y_max, res)
        X, Y = np.meshgrid(x, y)
        Z = np.vectorize(f)(X, Y, func_type)
        return X, Y, Z
    
    # UI Components
    main_ui = widgets.VBox()
    
    # Function selection
    function_dropdown = widgets.Dropdown(
        options=[
            ('Paraboloid', 'paraboloid'),
            ('Saddle', 'saddle'),
            ('Wave', 'wave')
        ],
        value='paraboloid',
        description='Function:'
    )
    
    # Point controls
    x_slider = widgets.FloatSlider(min=-3, max=3, value=1, step=0.1, description='x:')
    y_slider = widgets.FloatSlider(min=-3, max=3, value=1, step=0.1, description='y:')
    preset_buttons = widgets.ToggleButtons(
        options=[
            ('Origin', (0, 0)),
            ('High Point', (2, 2)),
            ('Low Point', (-2, -2))
        ],
        description='Presets:'
    )
    
    # Visualization controls
    resolution_slider = widgets.IntSlider(min=10, max=100, value=30, step=10, description='Resolution:')
    view_controls = widgets.VBox([
        widgets.IntSlider(min=-180, max=180, value=45, description='Azimuth:'),
        widgets.IntSlider(min=-90, max=90, value=30, description='Elevation:'),
        widgets.FloatSlider(min=0.1, max=1, value=0.5, description='Surface Opacity:')
    ])
    
    # Tabs for different views
    tabs = widgets.Tab()
    tabs.children = [
        widgets.Output(),  # 3D View
        widgets.Output(),  # 2D Sections
        widgets.Output(),  # Derivatives
        widgets.Output()   # Applications
    ]
    tabs.set_title(0, '3D View')
    tabs.set_title(1, '2D Slices')
    tabs.set_title(2, 'Derivatives')
    tabs.set_title(3, 'Applications')
    
    # Information display
    info_output = widgets.Output()
    
    # Layout organization
    controls = widgets.Accordion([
        widgets.VBox([function_dropdown, resolution_slider]),
        widgets.VBox([x_slider, y_slider, preset_buttons]),
        view_controls
    ])
    controls.set_title(0, 'General Settings')
    controls.set_title(1, 'Point Controls')
    controls.set_title(2, 'View Settings')
    
    # Debounced update mechanism
    def debounced_update(*args):
        time.sleep(0.3)
        update_all()
    
    # Update all views
    def update_all():
        # Common parameters
        x = x_slider.value
        y = y_slider.value
        res = resolution_slider.value
        func_type = function_dropdown.value
        
        # Generate data
        X, Y, Z = generate_data(-3, 3, -3, 3, res, func_type)
        f_val = f(x, y, func_type)
        df_dx_val = df_dx(x, y, func_type)
        df_dy_val = df_dy(x, y, func_type)
        
        # Update information panel
        with info_output:
            info_output.clear_output()
            print(f"Ponto: ({x:.2f}, {y:.2f})")
            print(f"f(x,y) = {f_val:.2f}")
            print(f"∂f/∂x = {df_dx_val:.2f}, ∂f/∂y = {df_dy_val:.2f}")
        
        # Update 3D View
        with tabs.children[0]:
            tabs.children[0].clear_output(wait=True)
            fig = go.Figure()
            
            # Surface plot
            fig.add_trace(go.Surface(
                x=X, y=Y, z=Z,
                colorscale='viridis',
                opacity=view_controls.children[2].value,
                showscale=False
            ))
            
            # Current point
            fig.add_trace(go.Scatter3d(
                x=[x], y=[y], z=[f_val],
                mode='markers',
                marker=dict(size=8, color='red'),
                name='Current Point'
            ))
            
            # Tangent plane
            x_range = np.linspace(-3, 3, 10)
            y_range = np.linspace(-3, 3, 10)
            X_plane, Y_plane = np.meshgrid(x_range, y_range)
            Z_plane = f_val + df_dx_val*(X_plane - x) + df_dy_val*(Y_plane - y)
            fig.add_trace(go.Surface(
                x=X_plane, y=Y_plane, z=Z_plane,
                colorscale='reds',
                opacity=0.5,
                showscale=False,
                name='Tangent Plane'
            ))
            
            fig.update_layout(
                scene=dict(
                    xaxis_title='X',
                    yaxis_title='Y',
                    zaxis_title='f(X,Y)',
                    camera=dict(
                        eye=dict(x=1.5, y=1.5, z=0.5),
                        center=dict(x=0, y=0, z=0),
                        projection=dict(type='orthographic')
                    )
                ),
                margin=dict(l=0, r=0, b=0, t=0)
            )
            fig.show()
        
        # Update 2D Slices
        with tabs.children[1]:
            tabs.children[1].clear_output(wait=True)
            fig = make_subplots(rows=1, cols=2)
            
            # X-slice
            x_vals = np.linspace(-3, 3, 100)
            z_x = np.array([f(xv, y, func_type) for xv in x_vals])
            fig.add_trace(go.Scatter(x=x_vals, y=z_x, name='Fix y'), row=1, col=1)
            fig.add_trace(go.Scatter(x=[x], y=[f_val], mode='markers', marker=dict(size=10, color='red')), row=1, col=1)
            
            # Y-slice
            y_vals = np.linspace(-3, 3, 100)
            z_y = np.array([f(x, yv, func_type) for yv in y_vals])
            fig.add_trace(go.Scatter(x=y_vals, y=z_y, name='Fix x'), row=1, col=2)
            fig.add_trace(go.Scatter(x=[y], y=[f_val], mode='markers', marker=dict(size=10, color='red')), row=1, col=2)
            
            fig.update_layout(height=500, width=1000, title_text="2D Slices")
            fig.show()
        
        # Update Derivatives
        with tabs.children[2]:
            tabs.children[2].clear_output(wait=True)
            fig = make_subplots(rows=1, cols=2)
            
            # df/dx
            t_vals = np.linspace(-3, 3, 100)
            dfdx_vals = np.array([df_dx(tv, 0, func_type) for tv in t_vals])
            fig.add_trace(go.Scatter(x=t_vals, y=dfdx_vals, name='∂f/∂x'), row=1, col=1)
            fig.add_trace(go.Scatter(x=[x], y=[df_dx_val], mode='markers', marker=dict(size=10, color='red')), row=1, col=1)
            
            # df/dy
            dfdy_vals = np.array([df_dy(0, tv, func_type) for tv in t_vals])
            fig.add_trace(go.Scatter(x=t_vals, y=dfdy_vals, name='∂f/∂y'), row=1, col=2)
            fig.add_trace(go.Scatter(x=[y], y=[df_dy_val], mode='markers', marker=dict(size=10, color='red')), row=1, col=2)
            
            fig.update_layout(height=500, width=1000, title_text="Partial Derivatives")
            fig.show()
        
        # Update Applications
        with tabs.children[3]:
            tabs.children[3].clear_output(wait=True)
            fig = go.Figure()
            
            days = np.linspace(30, 0, 100)
            price = 5 * np.exp(-0.05 * (30 - days)) + 2 * np.sqrt(days/30)
            fig.add_trace(go.Scatter(x=days, y=price, name='Option Price'))
            
            fig.update_layout(title='Theta (Time Decay)', xaxis_title='Days to Expiry', yaxis_title='Price')
            fig.show()
    
    # Widget interactions
    def set_preset(change):
        if change['type'] == 'change' and change['name'] == 'value':
            x, y = change['new']
            x_slider.value = x
            y_slider.value = y
    
    preset_buttons.observe(set_preset)
    
    # Connect widgets
    x_slider.observe(debounced_update, names='value')
    y_slider.observe(debounced_update, names='value')
    resolution_slider.observe(debounced_update, names='value')
    function_dropdown.observe(debounced_update, names='value')
    view_controls.children[0].observe(debounced_update, names='value')
    view_controls.children[1].observe(debounced_update, names='value')
    view_controls.children[2].observe(debounced_update, names='value')
    
    # Initial update
    update_all()
    
    # Final layout
    main_ui.children = [
        widgets.HBox([controls, tabs]),
        info_output
    ]
    
    display(main_ui)

# Run the demonstration
demonstrate_partial_derivative()